In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt

# 데이터 로드
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/monunmon.csv'
data = pd.read_csv(file_path)

# 데이터 확인
print("데이터 샘플:")
print(data.head())

# 특징(X)과 레이블(y) 분리
X = data.iloc[:, :-1]  # 특징 (feature)들
y = data.iloc[:, -1]   # 레이블 (0~94)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
데이터 샘플:
   Total Packets  Incoming Packets  Outgoing Packets  Incoming Ratio  \
0         1421.0             121.0            1300.0        0.085151   
1          518.0              80.0             438.0        0.154440   
2         1358.0             118.0            1240.0        0.086892   
3         1446.0             122.0            1324.0        0.084371   
4         1406.0             115.0            1291.0        0.081792   

   Outgoing Ratio  Outgoing Std  Outgoing Mean  Packets per Second  \
0        0.914849    515.483953     773.322314          140.138067   
1        0.845560    139.231951     226.162500           50.984252   
2        0.913108    472.735508     786.110169          122.232223   
3        0.915629    513.916038     820.139344          108.233533   
4        0.918208    503.993490     789.608696          132.142857   

   First 

In [15]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 레이블을 1과 0로 변환(monitored label:1/ unmonitored label:0)
data['Label'] = data['Label'].apply(lambda x: 1 if x != -1 else 0)

# feature와 label 분리
X = data.drop('Label', axis=1)  # 'Label'을 제외한 모든 열이 feature
y = data['Label']  # 'Label' 열이 target

# 학습용 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 데이터 형식으로 변환
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

**튜닝을 하지 않은 기본 LightGBM model**

In [19]:
# 기본 모델 파라미터 설정
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'random_state': 42
}

# 모델 학습 (조기 종료 없이)
print("Training LightGBM model...")
model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,  # 부스팅 단계 개수
    valid_sets=[test_data],  # 검증 데이터 설정

)

# 예측
y_pred_prob = model.predict(X_test, num_iteration=model.best_iteration)
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (Basic Model): {accuracy:.4f}')

Training LightGBM model...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 17600, number of negative: 0
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001681 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3345
[LightGBM] [Info] Number of data points in the train set: 17600, number of used features: 15
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Info] Start training from score 34.539576
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[Light

LIgtGBM 자꾸 모델 학습하는데 오류가 나는 부분이 있어서(early_Stopping/ verbose_eval)이 부분들을 삭제하고 모델을 돌리니 Accuracy가 1이 나오는데 이상한것 같아서 다른 팀원들과 더블 체킹이 필요할 것 같다...